# BJ's Capstone Project Team 18
### Paula Demacker, Jingjing Lu, Qiqi Tang, Xinping Yu

Web scraping note book



Imports


In [4]:
#imports
import selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd
import time

Test Chromer Driver  
uncomment the cell below to test your chrome driver 

In [1]:
# webdriver.Chrome() will be used
#driver = webdriver.Chrome()

#choose an URL
#url = "https://www.bjs.com/"
   
#opening link in the browser
#driver.get(url)

### Load the CSV file with location URLS and additional information we got from BJ's

In [7]:
df = pd.read_csv("urls_location_new.csv")
df = pd.DataFrame(df)
df

,Company,Address,City,State,Zip Code,Longitude,Latitude,URL
0,BJ's Wholesale,278 Middlesex Avenue,Medford,MA,2155,-71.082800,42.413370,https://www.google.com/maps/place/BJ's+Wholesa...
1,BJ's Wholesale,Northway Mall 1440 Central Ave.,Albany,NY,12205,-73.830730,42.704230,https://www.google.com/maps/place/BJ's+Wholesa...
2,BJ's Wholesale,300 Route 17 (Northbound),E. Rutherford,NJ,7073,-74.096790,40.824000,https://www.google.com/maps/place/BJ's+Wholesa...
3,BJ's Wholesale,6924 Frank Avenue Nw,N. Canton,OH,44720,-81.439790,40.881800,https://www.google.com/maps/place/BJ's+Wholesa...
4,BJ's Wholesale,4000 Nesconset Highway,E. Setauket,NY,11733,-73.076560,40.904170,https://www.google.com/maps/place/BJ's+Wholesa...
...,...,...,...,...,...,...,...,...
600,Walmart,29176 Van Dyke Avenue,Warren,MI,48093,-83.026566,42.508609,https://www.google.com/maps/place/Walmart+Supe...
601,Walmart,14900 Dix Toledo Rd,Southgate,MI,48195,-83.210720,42.201638,https://www.google.com/maps/place/Walmart+Supe...
602,Walmart,45400 Marketplace Blvd,Chesterfield,MI,48051,-82.852857,42.634760,https://www.google.com/maps/place/Walmart+Supe...
603,Walmart,1201 Route 300,Newburgh,NY,12550,-74.074669,41.497821,https://www.google.com/maps/place/Walmart+Supe...


Checking for missing values - no missing values in our data set

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Company    605 non-null    object 
 1   Address    605 non-null    object 
 2   City       605 non-null    object 
 3   State      605 non-null    object 
 4   Zip Code   605 non-null    object 
 5   Longitude  605 non-null    float64
 6   Latitude   605 non-null    float64
 7   URL        605 non-null    object 
dtypes: float64(2), object(6)
memory usage: 37.9+ KB


Write a for loop to extract all url from the dataframe

In [8]:
# empty list to store urls in
urls = []

# for loop to iterate over the urls and add them to the empty list
for u in df["URL"]:
    urls.append(u)

# confirm that all urls are in the list
len(urls)


605

In [9]:
companies_urls = pd.DataFrame(df,columns=['Company','URL'])
companies_urls


,Company,URL
0,BJ's Wholesale,https://www.google.com/maps/place/BJ's+Wholesa...
1,BJ's Wholesale,https://www.google.com/maps/place/BJ's+Wholesa...
2,BJ's Wholesale,https://www.google.com/maps/place/BJ's+Wholesa...
3,BJ's Wholesale,https://www.google.com/maps/place/BJ's+Wholesa...
4,BJ's Wholesale,https://www.google.com/maps/place/BJ's+Wholesa...
...,...,...
600,Walmart,https://www.google.com/maps/place/Walmart+Supe...
601,Walmart,https://www.google.com/maps/place/Walmart+Supe...
602,Walmart,https://www.google.com/maps/place/Walmart+Supe...
603,Walmart,https://www.google.com/maps/place/Walmart+Supe...


In [10]:
df['Company'].unique()

array(["BJ's Wholesale\xa0", 'Costco', "Sam's Club", 'Walmart'],
      dtype=object)

In [11]:
bj_wholesale = []
costco = []
sam_club = []
walmart = []
for (url, company) in zip(companies_urls['URL'], companies_urls['Company']):
    if company == "BJ's Wholesale\xa0":
        bj_wholesale.append(url)
    elif company == "Costco":
        costco.append(url)
    elif company == "Sam's Club":
        sam_club.append(url)
    else:
        walmart.append(url)



### Write code for the web scraping part 
 Define a class with attributed to get all the necessary information from Google Reviews

In [18]:
class WebDriver():

    location_data = {}
    review_data = {}

    def __get_driver(self, debug=False):
        options = Options()

        if not self.debug:
            options.add_argument("--headless")
        else:
            options.add_argument("--window-size=1366,768")

        options.add_argument("--disable-notifications")
        options.add_argument("--lang=en-GB")
        input_driver = webdriver.Chrome("C:/Program Files (x86)/Google/Chrome/Application/chromedriver",chrome_options=options)

        return input_driver


    def __init__(self):
        self.PATH = "C:/Program Files (x86)/Google/Chrome/Application/chromedriver"
        self.driver = webdriver.Chrome(self.PATH)
        self.location_data["average rating"] = []
        self.location_data["reviews count"] = []
        self.location_data["location"] = []       
        self.review_data["star rating"] = []
        self.review_data["name"] = []
        self.review_data["review"] = []
        self.review_data["date"] = []
        



    def click_all_reviews(self):
        print("Click all reviews...")

        try: 
            WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "widget-pane-link")))

            element = self.driver.find_element_by_css_selector("[class='widget-pane-link']")
            element.click()

        except:
            print("Click all reviews failed.")
            self.driver.quit()
            return False

        return True

    def get_location(self):
        print("Getting location data...")

        try:
            avg_rating = self.driver.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d")
            total_reviews = self.driver.find_element_by_class_name("widget-pane-link")
            address = self.driver.find_element_by_css_selector("[data-item-id='address']")
            
        except:
            print("No location found.")
            pass

        try:
            self.location_data["average rating"].append(avg_rating.text)
            self.location_data["reviews count"].append(total_reviews.text[0:-1])
            self.location_data["location"].append(address.text)

        except:
            print("Rating and Review Count failed.")
            pass

    def scroll_the_page(self):
        print("Scrolling down the page...")
        for i in range(700):
            js = "var q=document.getElementsByClassName('section-layout section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc')[0].scrollTop =600000"
            self.driver.execute_script(js)
            time.sleep(2)
            command_to_js = "return document.getElementsByClassName('section-layout section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc')[0].scrollHeight"
            Height =  self.driver.execute_script(command_to_js)
            print("------------------------------")
            print(Height)         
  

    def get_reviews(self):
        print("Getting reviews...")
        try:
            review_names = self.driver.find_elements_by_class_name("ODSEW-ShBeI-title")
            review_text = self.driver.find_elements_by_class_name("ODSEW-ShBeI-text")
            review_dates = self.driver.find_elements_by_css_selector("[class='ODSEW-ShBeI-RgZmSc-date']")
            review_stars = self.driver.find_elements_by_css_selector("[class='ODSEW-ShBeI-H1e3jb']")

            review_stars_final = []

            for i in review_stars:
                review_stars_final.append(i.get_attribute("aria-label"))

            review_names_list = [a.text for a in review_names]
            review_text_list = [a.text for a in review_text]
            review_dates_list = [a.text for a in review_dates]
            review_stars_list = [a for a in review_stars_final]
            print(len(review_stars_list))

            for (a,b,c,d) in zip(review_names_list, review_text_list, review_dates_list, review_stars_list):
                self.review_data["name"].append(a)
                self.review_data["review"].append(b)
                self.review_data["date"].append(c)
                self.review_data["star rating"].append(d)

        except:
            print("Getting reviews failed.")
            pass

    def scrape(self, url):
        print("Scarping...")
        try:
            self.driver.get(url)

        except:
            print("Scraping failed.")
            pass
        time.sleep(3)

        self.get_location()
        if(self.click_all_reviews()==False):
            pass
        time.sleep(4)
        self.scroll_the_page()
        self.get_reviews()
        self.driver.quit()

        return(self.location_data, self.review_data)


Scraping just one URL at the time and testing if our Web Driver class works

In [11]:
url = "https://www.google.com/maps/place/BJ's+Wholesale+Club/@42.413148,-71.0843377,17z/data=!3m1!4b1!4m5!3m4!1s0x89e3713f4df53f97:0x7bbe47e5e3361afc!8m2!3d42.413148!4d-71.082149"
x = WebDriver()
bjs = x.scrape(url)
print(bjs)


Scarping...
Getting location data...
Click all reviews...
Scrolling down the page...
------------------------------
6586
------------------------------
9509
------------------------------
12193
------------------------------
14669
------------------------------
16943
------------------------------
19095
------------------------------
21123
------------------------------
23172
------------------------------
25159
------------------------------
27166
------------------------------
29173
------------------------------
31763
------------------------------
34556
------------------------------
36318
------------------------------
37688
------------------------------
39058
------------------------------
40428
------------------------------
41798
------------------------------
43168
------------------------------
43168
------------------------------
44538
------------------------------
45908
------------------------------
45908
------------------------------
47278
-----------------------------

Creating dataframes we can then use for the scraping and fill in with the information we got from scraping

In [19]:
bj_reviews = pd.DataFrame({"star rating": [], "name": [], "review": []})
bj_overview_location = pd.DataFrame({"average rating": [], "reviews count": [], "location":[]})
y = 1



Scraping all BJ's URLs with a for loop  
Attention: If you try to do it again, it is better to only scrape 20-30 urls at a time since google will detect you as a robot and kick you out

In [20]:
for url in bj_wholesale:
    
    x=WebDriver()
    bjs = x.scrape(url)
    overview_location = bjs[0]
    reviews = bjs[1]

    print("Append to Dataframes...")

    bj_reviews2 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in reviews.items() ]))
    bj_reviews2['location_code'] = y
    bj_overview_location2 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in overview_location.items() ]))
    bj_overview_location2['location_code'] = y
    bj_reviews = pd.concat([bj_reviews,bj_reviews2], ignore_index = True)
    bj_overview_location = pd.concat([bj_overview_location,bj_overview_location2], ignore_index = True)

    y+=1

    print(bj_reviews, bj_overview_location)
    time.sleep(3)
    

Scarping...
Getting location data...
No location found.
Rating and Review Count failed.
Click all reviews...
Scrolling down the page...


JavascriptException: Message: javascript error: Cannot set property 'scrollTop' of undefined
  (Session info: chrome=89.0.4389.128)


Save scraped information in a csv for later

In [18]:
  bj_reviews.to_csv(r'C:/Users/30842/OneDrive/桌面/Capstone/bjs_reviews_80_120.csv')
  bj_overview_location.to_csv(r'C:/Users/30842/OneDrive/桌面/Capstone/bjs_location_overview_80_120.csv')
